Get dialogues

In [122]:
import json
import re
import pandas as pd
import copy

Slots

# 1.Собираем фразы из диалогов.

In [26]:
with open('/content/dialogues_012_russian.json','r',encoding = 'utf-8') as f:
  dialogues = json.load(f)

In [123]:
def get_utts(dialog_number):
  utts = []
  i = -1
  for turn in dialogues[dialog_number]['turns']:    
    i += 1
    ut = dialogues[dialog_number]['turns'][i]['utterance']
    utts.append(ut)
  return utts

In [124]:
get_utts(4)

[' Ищу информацию. Вы можете помочь мне с местом, где я могу остановиться, пока я буду в городе?',
 ' Я могу помочь вам. В какой части города?',
 ' Я хочу поехать в северную часть города, там есть бесплатная парковка и 4 звезды.',
 ' Хорошо, а как насчет гостевого дома Acorn? Он расположен на 154 честертон-роуд. Вы хотите, чтобы я сделал заказ?',
 ' Это звучит идеально. Мне нужна комната на 3 человека и 4 ночи, начиная с воскресенья.',
 ' Бронирование прошло успешно. Вы забронировали номер для 3 человек, остановившись на 4 ночи в гостевом доме Acorn, начиная с воскресенья. Ваш справочный номер: Z45I87LD.',
 ' Спасибо! Еще мне нужен бассейн в северной части Кембриджа.',
 ' Я бы порекомендовал Иисус зеленый открытый бассейн. Хотите узнать больше?',
 ' Не могли бы вы сообщить мне адрес, почтовый индекс и номер телефона?',
 ' Конечно! Адрес находится между дорогой Виктория и рекой, почтовый индекс cb43px, номер телефона 01223302579.',
 ' Напоследок такси от отеля до бассейна, пожалуйста.',

# 2. Достаем слоты из размеченных диалогов

In [125]:
with open('/content/dialogues_012_ru.txt','r',encoding = 'utf-8') as f:
  dialogues_txt = f.read().split('\n\n')

In [153]:
def get_dialogue(dialogue_number):
  dialogue = dialogues_txt[dialogue_number].split("\n")
  return dialogue

In [155]:
len(dialogues_txt)

513

In [127]:
dialogu_e = get_dialogue(4)

In [128]:
dialogue_4 = []
for x in dialogu_e:
  if len(x)>0:
    dialogue_4.append(x)

In [129]:
dialogue_4

['Ищу информацию. Вы можете помочь мне с местом, где я могу остановиться, пока я буду в городе?',
 'Я могу помочь вам. В какой части города?',
 'Я хочу поехать в северную часть города, там есть бесплатная парковка и 4 звезды.',
 'Хорошо, а как насчет [гостевого дома Acorn]? Он расположен на 154 честертон-роуд. Вы хотите, чтобы я сделал заказ?',
 'Это звучит идеально. Мне нужна комната на 3 человека и 4 ночи, начиная с воскресенья.',
 'Бронирование прошло успешно. Вы забронировали номер для 3 человек, остановившись на 4 ночи в гостевом доме Acorn, начиная с воскресенья. Ваш справочный номер: Z45I87LD.',
 'Спасибо! Еще мне нужен бассейн в северной части Кембриджа.',
 'Я бы порекомендовал [Иисус зеленый открытый бассейн]. Хотите узнать больше?',
 'Не могли бы вы сообщить мне адрес, почтовый индекс и номер телефона?',
 'Конечно! Адрес находится между дорогой Виктория и рекой, почтовый индекс cb43px, номер телефона 01223302579.',
 'Напоследок такси от отеля до бассейна, пожалуйста.',
 'Вы д

## 2.1 Достаем слоты из диалога

In [130]:
def get_slots(dialogue_number):
  slots_russian = re.findall(r'\[.+\]', dialogues_txt[dialogue_number])
  return slots_russian

In [131]:
get_slots(4)

['[гостевого дома Acorn]',
 '[Иисус зеленый открытый бассейн]',
 '[10:30]',
 '[17:00]']

# 3. Датафрейм для замены слотов

In [132]:
worksheets_dfs = pd.DataFrame()
xlsx = pd.ExcelFile('/content/slot_replacements.xlsx')
sheets = xlsx.sheet_names
for sheet in sheets[1:3]: # Достаем необходимые листы для склейки в датафрейм ('attraction','hotels')
  data = xlsx.parse(sheet_name = sheet)
  worksheets_dfs = worksheets_dfs.append(data)
worksheets_dfs.to_excel('replacement_combined.xlsx')

# 4. Замена слотов

## 4.1 Достаем индекс

In [133]:
# Открываем replacement_combined.xlsx
replacement_combined = []
content = pd.ExcelFile('/content/replacement_combined.xlsx')
sheets = content.sheet_names
for sheet in sheets:
  data = pd.read_excel(content, sheet_name = sheet)
  data["sheet"] = sheet
  replacement_combined.append(data)

In [134]:
### достаем все индексы из таблички
inds = []
for slot in range(len(get_slots(4))):
  for line in replacement_combined[0]['ru']:
    if get_slots(4)[slot] in str(line):
      index = replacement_combined[0].index
      condition = replacement_combined[0]['ru'] == line
      slot_indices = index[condition]
      slot_indices_list = slot_indices.tolist()
      ind = slot_indices_list
      inds.append(ind)
print(inds)

[[760], [92]]


In [135]:
# Достаем сущности по индексам
final_slots_russian = []
for ind in inds:
  final_slots_russian.append(' '.join(replacement_combined[0]['repl'][ind].tolist()))

In [136]:
final_slots_russian 

["'[гостевого дома Величъ]'", "'[Открытый Аквакомплекс «Лужники»]'"]

In [137]:
# Создаем новый список слотов
slots = get_slots(4)
i = 0
for slot in final_slots_russian:
  slots[i] = slot
  i += 1

In [138]:
slots

["'[гостевого дома Величъ]'",
 "'[Открытый Аквакомплекс «Лужники»]'",
 '[10:30]',
 '[17:00]']

In [139]:
# пересобираем текстовые данные
utts = dialogue_4
i = 0
for utt in utts:
  if re.findall(r'\[.+\]', utt):
    ind = utts.index(utt)
    utts[ind] =  re.sub(r'\[.+\]', slots[i], utts[ind])
    i += 1
print(utts)

['Ищу информацию. Вы можете помочь мне с местом, где я могу остановиться, пока я буду в городе?', 'Я могу помочь вам. В какой части города?', 'Я хочу поехать в северную часть города, там есть бесплатная парковка и 4 звезды.', "Хорошо, а как насчет '[гостевого дома Величъ]'? Он расположен на 154 честертон-роуд. Вы хотите, чтобы я сделал заказ?", 'Это звучит идеально. Мне нужна комната на 3 человека и 4 ночи, начиная с воскресенья.', 'Бронирование прошло успешно. Вы забронировали номер для 3 человек, остановившись на 4 ночи в гостевом доме Acorn, начиная с воскресенья. Ваш справочный номер: Z45I87LD.', 'Спасибо! Еще мне нужен бассейн в северной части Кембриджа.', "Я бы порекомендовал '[Открытый Аквакомплекс «Лужники»]'. Хотите узнать больше?", 'Не могли бы вы сообщить мне адрес, почтовый индекс и номер телефона?', 'Конечно! Адрес находится между дорогой Виктория и рекой, почтовый индекс cb43px, номер телефона 01223302579.', 'Напоследок такси от отеля до бассейна, пожалуйста.', 'Вы думает

In [140]:
dialogue_4

['Ищу информацию. Вы можете помочь мне с местом, где я могу остановиться, пока я буду в городе?',
 'Я могу помочь вам. В какой части города?',
 'Я хочу поехать в северную часть города, там есть бесплатная парковка и 4 звезды.',
 "Хорошо, а как насчет '[гостевого дома Величъ]'? Он расположен на 154 честертон-роуд. Вы хотите, чтобы я сделал заказ?",
 'Это звучит идеально. Мне нужна комната на 3 человека и 4 ночи, начиная с воскресенья.',
 'Бронирование прошло успешно. Вы забронировали номер для 3 человек, остановившись на 4 ночи в гостевом доме Acorn, начиная с воскресенья. Ваш справочный номер: Z45I87LD.',
 'Спасибо! Еще мне нужен бассейн в северной части Кембриджа.',
 "Я бы порекомендовал '[Открытый Аквакомплекс «Лужники»]'. Хотите узнать больше?",
 'Не могли бы вы сообщить мне адрес, почтовый индекс и номер телефона?',
 'Конечно! Адрес находится между дорогой Виктория и рекой, почтовый индекс cb43px, номер телефона 01223302579.',
 'Напоследок такси от отеля до бассейна, пожалуйста.',


In [141]:
def parse_translate(translated_text):
    lines = translated_text.split('\n')
    texts = []
    current_text = []
    skip = False
    for line in lines:
        if not skip:
            if line != ' ':
                current_text.append(line)
            else:
                texts.append(current_text)
                skip = True
        else:
            skip = False
            current_text = []

    return texts

In [142]:
translated = dialogue_4

In [143]:
def translated2json(translated_dialogue, dialogue):
    new_dialogue = dialogue.copy()
    for i, text in enumerate(translated_dialogue):
        spans = list(reversed(search_spans(text, "[", "]"))) # reverse list, the first entry become last
        offset_front = 0
        offset_back = 1
        remove_spec_brackets = False
        for frame in new_dialogue['turns'][i]['frames']:
            for slot in frame['slots']:
                if 'start' in slot:
                    if len(spans) != 0:
                        start, end = spans.pop() # after reverse, we can use .pop() to get and remove first entry (i.e 'spans' list is now FIFO-queue)
                    elif "«" in text:
                        start, end = search_spans(text, "«", "»")[0] # assuming that there is only one span with «» brackets
                        remove_spec_brackets = True
                    else:
                        continue
                        
                    slot['start'] = start - offset_front
                    slot['exclusive_end'] = end - offset_back # offsetting '[', which will be removed
                    slot['value'] = text[start + 1:end] # offsetting not yet remove '[' for value extraction
                    offset_front += 2
                    offset_back += 2
                        

        new_text = text.replace("[", "").replace("]", "")
        if remove_spec_brackets:
              new_text = new_text.replace("«", "").replace("»", "")
        new_dialogue['turns'][i]['utterance'] = new_text
            
        #print(new_dialogue['turns'][i]['utterance'])
    return new_dialogue

In [144]:
def search_spans(text, start_symbol: str = "[", end_symbol: str = "]"):
    subtext1=start_symbol
    result1 = []
    flag = False
    n = 1
    for i,element in enumerate(text):
        if flag:
            if n > len(subtext1)-1:
                flag = False
                n = 1
            elif element == subtext1[n]:
                n += 1
            elif element != subtext1[n]:
                result1 = result1[:-1]
                n = 1
                flag = False
        if element == subtext1[0] and not flag:
            flag = True
            result1.append(i)

    subtext2=end_symbol
 
    result2 = []

    flag = False
    n = 1

    for i,element in enumerate(text):
        if flag:
            if n > len(subtext2)-1:
                flag = False
                n = 1
            elif element == subtext2[n]:
                n += 1
            elif element != subtext2[n]:
                result2 = result2[:-1]
                n = 1
                flag = False
        if element == subtext2[0] and not flag:
            flag = True
            result2.append(i)
    ress = list(zip(result1, result2))

    return ress

In [145]:
!wget https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_012.json

--2022-03-22 22:35:05--  https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_012.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10584880 (10M) [text/plain]
Saving to: ‘dialogues_012.json.1’

dialogues_012.json. 100%[===================>]  10.09M  --.-KB/s    in 0.04s   

2022-03-22 22:35:05 (277 MB/s) - ‘dialogues_012.json.1’ saved [10584880/10584880]



In [156]:
with open("/content/dialogues_012.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [157]:
data[4]

{'dialogue_id': 'PMUL3490.json',
 'services': ['taxi', 'attraction', 'hotel'],
 'turns': [{'frames': [{'actions': [],
     'service': 'taxi',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'train',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'bus',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'police',
     'slots': [],
     'state': {'active_intent': 'NONE',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'hotel',
     'slots': [],
     'state': {'active_intent': 'find_hotel',
      'requested_slots': [],
      'slot_values': {}}},
    {'actions': [],
     'service': 'restaurant',
     'slots': [],
     'state': {'active_intent': 'NO

In [148]:
res = translated2json(translated, data[4])

In [ ]:
res

In [150]:
translated_json = []
for i in range(len(translated)):
    translated_json.append(res)
    
with open("dialogues_001_russian.json", "w", encoding='utf-8') as f:
    json.dump(translated_json, f, indent=2, ensure_ascii=False)